In [1]:
import numpy as np
import pandas as pd

# IDK, but importing BERTopic without this ends with error
from torch import device

from bertopic import BERTopic
import datamapplot

import gc
import pickle
import os
os.chdir("/home/denisalpino/dev/FinABYSS/")

2025-05-25 10:35:52.393735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748158552.544749    2038 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748158552.595326    2038 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748158552.940464    2038 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748158552.940510    2038 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748158552.940512    2038 computation_placer.cc:177] computation placer alr

In [2]:
embeddings_2d = np.load("embeddings2d.npy")
labels = np.load("labels.npy")

In [3]:
embeddings_full = np.load('data/preprocessed/embeddings_mp.npy')
rows, _ = embeddings_full.shape

In [4]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
embeddings_idx = np.random.choice(rows, size=int(rows * 0.10), replace=False)
embeddings = embeddings_full[embeddings_idx]

In [ ]:
del embeddings_full
gc.collect();

In [6]:
corpus = pd.read_parquet("data/preprocessed/articles.parquet")
df = corpus.iloc[embeddings_idx].reset_index(drop=True)

del corpus, embeddings
gc.collect()

0

In [7]:
topic_model = BERTopic.load("topic_model")

2025-05-25 10:37:47,194 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


### Preprocessing

In [8]:
# Номер кластера для каждой новости
topic_per_doc = topic_model.topics_

unique_topics = set(topic_per_doc)

names = [topic_model.custom_labels_[topic + topic_model._outliers] for topic in unique_topics]
topic_name_mapping = {topic_num: topic_name for topic_num, topic_name in zip(unique_topics, names)}

# Меняем название выбросов для совместимоси с datamapplot
topic_name_mapping[-1] = "Unlabelled"

# Массив названиий кластера для каждой новости
named_topic_per_doc = pd.Series(topic_per_doc).map(topic_name_mapping).values


data = []
for x, y, dt, label, text, title, source, tickers, url in zip(
        embeddings_2d[:, 0],
        embeddings_2d[:, 1],
        df["datetime"],
        named_topic_per_doc,
        df["text"],
        df["title"],
        df["source"],
        df["assets"],
        df["url"]
    ):
    date, time = dt.strftime("%d.%m.%Y %H:%M:%S").split()
    #if label != "Unlabelled":
    row = [
        x, y, title, label, f"{title}\n\nTopic: {label}\nDate: {date}\nTime: {time}\nTickers: {', '.join(tickers)}\nSource: {source}",
        dt, date + " " + time, len(text), url, ', '.join(tickers), source
    ]
    data.append(row)

df2 = pd.DataFrame(data, columns=["x", "y", "title", "topic", "hover_text", "datetime", "datetime_formatted", "char_len", "link", "tickers", "source"])
df2["datetime"] = pd.to_datetime(df2["datetime"])

In [ ]:
del df, data, named_topic_per_doc, topic_name_mapping, names, topic_per_doc
gc.collect()

0

In [10]:
# ================================== #
# ДЛЯ РАМОК ВНУТРИ ВСПЛЫВАЮЩЕГО ОКНА #
# ================================== #
badge_css = """
    border-radius:6px;
    width:fit-content;
    max-width:75%;
    margin:2px;
    padding: 2px 10px 2px 10px;
    font-size: 10pt;
"""

hover_text_template = """
<div>
    <p>{title}</p>
    <div style="background-color:{{color}};color:#fff;{badge_css}">Source: {{source}}</div>
    <div style="background-color:#eeeeeeff;{badge_css}">Published: {{dt}}</div>
</div>
"""

# Use with the next parameters:
#   extra_point_data=extra_data,
#   hover_text_html_template=hover_text_template,

# =============================== #
# ДЛЯ ВЫБОРА ИСТОЧНИКОВ В ЛЕГЕНДЕ #
# =============================== #
custom_css="""
.row {
    display : flex;
    align-items : center;
}
.box {
    height:10px;
    width:10px;
    border-radius:2px;
    margin-right:5px;
    padding:0px 0 1px 0;
    text-align:center;
    color: white;
    font-size: 14px;
    cursor: pointer;
}
#legend {
    position: absolute;
    top: 0;
    right: 0;
}
#title-container {
    max-width: 75%;
}

"""
custom_html = """
<div id="legend" class="container-box">
"""

sources = df["source"].unique() # Нужно создать список ресурсов
for source in sources:
    custom_html += f'    <div class="row"><div id="{source}"' + 'class="box" style="background-color:{#000000};">' + f'</div>{source}</div>\n'

custom_html += """
</div>
"""


custom_js = """
const legend = document.getElementById("legend");
const selectedPrimaryFields = new Set();

legend.addEventListener('click', function (event) {
    const selectedField = event.srcElement.id;

    if (selectedField) {
        if (selectedPrimaryFields.has(selectedField)) {
            selectedPrimaryFields.delete(selectedField);
            event.srcElement.innerHTML = "";
        } else {
            selectedPrimaryFields.add(selectedField);
            event.srcElement.innerHTML = "✓";
        }
    }
    const selectedIndices = [];
    datamap.metaData.primary_field.forEach((source, i) => {
        if (selectedPrimaryFields.has(source)) {
            selectedIndices.push(i);
        }
    });
    datamap.addSelection(selectedIndices, "legend");
});
"""
#  Use with the next parameters:
#   custom_css=custom_css,
#   custom_html=custom_html,
#   custom_js=custom_js

In [27]:
badge_css = """
    border-radius:6px;
    width:fit-content;
    max-width:75%;
    margin:2px;
    padding: 2px 10px 2px 10px;
    font-size: 10pt;
"""
hover_text_template = f"""
<div>
    <div style="font-size:12pt;padding:2px;"><b>{{title}}</b></div>
    <div style="background-color:#0000FF;{badge_css}">{{topic}}</div>
    <div style="background-color:#FF0000;{badge_css}">{{tickers}}</div>
    <div>SOURCE: {{source}}</div>
    <div>{{datetime_formatted}}</div>
</div>
"""
custom_css_logo = """
#logo-container {
    position: absolute;
    right: 0;
    top: 0;
    margin: 8px 16px 8px 16px;
    z-index: 0;
}
"""
custom_html = """
<div id="logo-container" class="container-box" style="position: fixed; z-index: 0; top: 0; right: 0;">
    <img src="file://wsl.localhost/Ubuntu/home/denisalpino/dev/FinABYSS/docs/finabyss_dark.svg" style="width:512px" />
</div>
"""

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl

# cmap = plt.get_cmap('plasma_r')

In [ ]:
custom_css="""
.row {
    display: flex;
    align-items: center;
    /* Чтобы название источника переносилось */
    overflow-wrap: break-word;
    white-space: normal;
}

.box {
    height: 10px;
    width: 10px;
    border-radius: 2px;
    margin-right: 5px;
    padding: 0 0 1px 0;
    text-align: center;
    color: white;
    font-size: 14px;
    cursor: pointer;
}

#legend {
    position: absolute;
    top: 0;
    right: 0;
    /* фиксированные размеры */
    height: 512px;
    width: 256px;
    /* когда контента больше, появляется скролл */
    overflow-y: auto;
    overflow-x: hidden;
    /* перенос слов внутри */
    word-wrap: break-word;
    white-space: normal;
}

#title-container {
    max-width: 100%;
}
"""

n_sources = len(df['source'].value_counts()[:50].index)
color_mapping = plt.get_cmap('inferno', n_sources)
colors = [
    mcolors.to_hex(color_mapping(i))    # cmap(i) возвращает RGBA,
    for i in range(n_sources)
]
color_mapping = dict(zip(df['source'].value_counts()[:50].index, colors))

# 3) Генерация HTML блока с легендой
custom_html = """
<div id="legend" class="container-box">
"""
for source, color in color_mapping.items():
    custom_html += f'    <div class="row"><div id="{source}" class="box" style="background-color:{color};"></div>{source}</div>\n'
custom_html += """
</div>
"""

cucustom_js = """
const legend = document.getElementById("legend");
const selectedSources = new Set();

legend.addEventListener('click', function(event) {
    const row = event.target.closest('.row');
    if (!row || !legend.contains(row)) return;

    const box = row.querySelector('.box');
    const src = box.id;

    if (selectedSources.has(src)) {
        selectedSources.delete(src);
        box.textContent = '';
    } else {
        selectedSources.add(src);
        box.textContent = '✓';
    }

    // Сбрасываем прошлые выделения для нашего класса
    datamap.clearSelection("legend");

    // Собираем индексы по реальному полю source
    const indices = datamap.metaData.source
        .map((s, i) => selectedSources.has(s) ? i : -1)
        .filter(i => i >= 0);

    if (indices.length > 0) {
        datamap.addSelection(indices, "legend");
    }
});

"""

In [28]:
plot = datamapplot.create_interactive_plot(
    df2[["x", "y"]].to_numpy(),
    df2["topic"],
    hover_text=df2["hover_text"], # Change to hover_text
    darkmode=True,

    color_label_text=False, # Названия кластеров не под цвет кластера, а просто белые
    # cmap=cmap,
    palette_hue_radius_dependence=10,
    cvd_safer=True,

    title="Semantic Map of Financial News",
    sub_title="Sample of 125,000 news articles from Yahoo! Finance",
    font_family="Montserrat",
    text_min_pixel_size=8,
    text_max_pixel_size=36,
    text_collision_size_scale=15,       # Чтобы заголовки тем не наезжали друг на друга
    use_medoids=True,                  # Для того, чтобы заголовки тем раcполагаели не в середине, а в медиане кластера

    # Выделяем кластеры
    polygon_alpha=2,
    cluster_boundary_polygons=True,
    cluster_boundary_line_width=1,

    # Устанавливаем величину точек относительно длины текста
    marker_size_array=df2["char_len"],
    point_radius_min_pixels=1.5,
    point_radius_max_pixels=3,
    point_line_width=0, # Убираем белый контур вокруг точек

    # Добавляем гистограму для фильтрации по количеству символов
    histogram_data=df2["datetime"],
    histogram_settings={
        "histogram_title":"Dates of Publication",
        "histogram_bin_fill_color":"#a64531",
        "histogram_bin_unselected_fill_color":"#d6a591",
        "histogram_bin_selected_fill_color":"#f68571",
        "histogram_width":400,
        "histogram_height":125,
    },

    # Поиск по скрытым текстам
    enable_search=True,
    # search_field="full_text",

    # Делаем точки кликабельными
    on_click="window.open(`{link}`)",
    extra_point_data=df2[["link", "title", "topic", "tickers", "source", "datetime_formatted"]],

    # Добавляем гистограму для фильтрации по количеству символов
    # selection_handler=datamapplot.selection_handlers.WordCloud(
    #     256,
    #     width=500,
    #     height=300,
    #     color_scale="viridis_r",
    #     n_rotations=0,
    #     use_idf=True
    # )
    #logo="file://wsl.localhost/Ubuntu/home/denisalpino/dev/FinABYSS/docs/finabyss_dark.svg",
    #logo_width=512,

    noise_label="Unlabelled",
    noise_color="#333333",

    initial_zoom_fraction=0.8,

    custom_css=custom_css_logo,
    custom_html=custom_html,
    # custom_js=custom_js,
    hover_text_html_template=hover_text_template,
)
plot.save("new_map.html")

In [ ]:
plot

In [9]:
sub = df2.sample(frac=0.5)

In [10]:
del df2
gc.collect()

0

In [ ]:
fig = datamapplot.create_plot(
    sub[["x", "y"]].to_numpy(),
    sub["topic"],
    darkmode=True,
    title="Semantic Map of Financial News",
    sub_title="News articles from Yahoo! Finance",
    use_medoids=False,
    cvd_safer=True,
    palette_hue_radius_dependence=20,
    figsize=(16, 16),
    label_over_points=False, # Названия будут отображаться не с боку
    dynamic_label_size=True, # Названия будут уменьшаться с размерм кластера
    dynamic_label_size_scaling_factor=0.4, # Сем ближе к 0, тем меньше различие
    max_font_size=16,
    min_font_size=6,
    font_family="Urbanist",
    min_font_weight=100,
    max_font_weight=400,

    dpi=50,
    pylabeladjust_adjust_by_size=False,
    force_matplotlib=True
    # highlight_labels=[
    #     "Cybersecurity",
    #     "Sustainable Finance",
    #     "Green Energy",
    #     "Topic modelling and text classification"
    # ],
    # highlight_label_keywords={"fontsize": 26, "fontweight": "black", "bbox":{"boxstyle":"round"}}
)

In [ ]:
fig